### Clean Title Function with LLM.

In [1]:
import transformers
import torch
from transformers import BitsAndBytesConfig, pipeline, AutoTokenizer
ACCESS_TOKEN = ""
device = "cuda:3"

def load_70b_model(device):
    model_id = "meta-llama/Meta-Llama-3.1-70B-Instruct"
        # Load model directly

    nf4_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    model_nf4 = transformers.AutoModelForCausalLM.from_pretrained(model_id, 
                                                    quantization_config=nf4_config,
                                                    device_map={"": device},
                                                    token=ACCESS_TOKEN)
    
    tokenizer = AutoTokenizer.from_pretrained(model_id, token=ACCESS_TOKEN)
    
    pipeline = transformers.pipeline(
        "text-generation",
        model= model_nf4, #model_id,
        # model_kwargs={"torch_dtype": torch.bfloat16},
        tokenizer=tokenizer,
        # device=device,
    )

    messages = [
        {"role": "system", "content": "You are an expert and experienced from the healthcare and biomedical domain with extensive medical knowledge and practical experience. Your name is OpenBioLLM, and your job is to annotate dictionary features and contexts learned by interpretability techniques. Please answer the below message."},
        {"role": "user", "content": "Hello?"},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
    )
    print(outputs[0]["generated_text"][len(prompt):])
    return pipeline

llama3 = load_70b_model(device)

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Hello. I'm OpenBioLLM, an expert in the healthcare and biomedical domain. I'm here to assist with annotating dictionary features and contexts learned by interpretability techniques. How can I help you today?


In [2]:
def generate_text_with_icl(prompt, pipeline, task_examples, max_new_tokens=256, temperature=0.00001, top_p=0.99) -> str:
    """
    Generate text using the specified prompt and parameters with in-context learning.
    Args:
    prompt (str): The input prompt for text generation.
    pipeline: The text generation pipeline.
    task_examples (list): List of dictionaries containing input-output pairs for in-context learning.
    max_new_tokens (int, optional): The maximum number of new tokens to generate. Defaults to 256.
    temperature (float, optional): The temperature value for sampling. Defaults to 0.00001.
    top_p (float, optional): The top-p value for sampling. Defaults to 0.99.
    Returns:
    str: The generated text.
    """
    # Construct the in-context learning prompt
    icl_prompt = "You are an expert and experienced from the healthcare and biomedical domain with extensive medical knowledge and practical experience. Your job is to help annotate specific tasks by looking for common patterns within text. Here are some examples of how to perform the task:\n\n"
    
    for example in task_examples:
        icl_prompt += f"Input: {example['input']}\nOutput: {example['output']}\n\n"
    
    icl_prompt += f"Now, please perform the same task for the following input:\nInput: {prompt}\nOutput:"

    messages = [
        {"role": "system", "content": icl_prompt},
        {"role": "user", "content": prompt},
    ]
    
    full_prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    
    outputs = pipeline(
        full_prompt,
        max_new_tokens=max_new_tokens,
        eos_token_id=terminators,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
    )
    
    return outputs[0]["generated_text"][len(full_prompt):]

In [4]:
title_cleaning_prompt = """
You are an assistant specialized in cleaning and standardizing academic paper titles. Your task is to take a given title and improve its formatting, spacing, and consistency. Follow these rules:

1. Correct spacing:
   - Ensure single spaces between words.
   - Remove extra spaces before or after hyphens.
   - Add spaces after colons and semicolons.

2. Hyphenation:
   - Use hyphens consistently in compound terms (e.g., "Multi-Scale" not "Multi Scale" or "MultiScale").
   - Correct common hyphenation errors in technical terms (e.g., "Pre-processing" not "Preprocessing").

3. Capitalization:
   - Use title case: Capitalize the first letter of each major word.
   - Do not capitalize articles (a, an, the), coordinating conjunctions (and, but, for, or, nor), or prepositions unless they start the title.
   - Always capitalize the first and last words of the title and subtitle.

4. Acronyms and initialisms:
   - Remove spaces between letters in acronyms (e.g., "CNN" not "C N N").
   - Ensure correct formatting of technical acronyms (e.g., "U-Net" not "UNet" or "U Net").

5. Special characters:
   - Correct the use of special characters like hyphens (-), en dashes (–), and em dashes (—).
   - Ensure proper use of quotation marks and apostrophes.

6. Consistency:
   - Maintain consistent formatting throughout the title.
   - Ensure that similar terms or concepts are formatted the same way.

7. Grammar and spelling:
   - Correct any obvious spelling errors.
   - Ensure proper grammatical structure.

8. No Authors: If the title contains any author names, emails, or affiliations, remove them.

When given a title, apply these rules to clean and standardize it. Provide the corrected title without additional commentary unless there are ambiguities or decisions that require explanation.

Title to clean: {title}

Cleaned title:
"""
import pandas as pd 
def clean_titles(pipeline, df : pd.DataFrame, prompt : str):
    cleaned_titles = []
    for title in df["title"]:
        cleaned_title = generate_text_with_icl(title, pipeline, [{"input": title, "output": ""}], max_new_tokens=256, temperature=0.00001, top_p=0.99)
        cleaned_titles.append(cleaned_title)
    return cleaned_titles



chil_info = pd.read_csv("processed_data/chil_extracted_info.csv") # get all the titles, and clean them.
chil_info.head(2)


,year,title,authors,abstract,code_count,gitlab_count,zenodo_count,dataset_count,mimic_count,eicu_count,...,tcga_count,gdc_count,seer_count,tuh_eeg_corpus_count,tuh_abnormal_eeg_corpus_count,tuh_eeg_artifact_corpus_count,tuh_eeg_epilepsy_corpus_count,tuh_eeg_events_corpus_count,tuh_eeg_seizure_corpus_count,tuh_eeg_slowing_corpus_count
0,2020,B M M- Net: Automatic Segmentationof Edemain O...,"Ruru Zhang, Beijing University of Posts and, T...",Retinal effusions and cysts caused by the leak...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020,Hurtful Words: Quantifying Biasesin Clinical C...,"Haoran Zhang∗, haoran@cs.toronto.edu, Universi...","In this work, we examine the extent to which e...",1,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
chil_info["authors"][0]

'Ruru Zhang, Beijing University of Posts and, Telecommunications, Haidian District, Beijing, China, zrr@bupt.edu.cnJiawen He, Beijing University of Posts and, Telecommunications, Haidian District, Beijing, China, euphy@bupt.edu.cnShenda Shi, Beijing University of Posts and, Telecommunications, Haidian District, Beijing, China, cy_z_feng@bupt.edu.cn, Haihong E, Beijing University of Posts and, Telecommunications'

In [12]:


def extract_emails(text, nlp):
    prompt = f"""
    Extract all email addresses from the following text. 
    Only output the email addresses, one per line. 
    If there are no email addresses, output 'No email addresses found. We are not using code here.'

    Text: {text}

    Extracted email addresses:
    """

    # Generate the response
    response = generate_text_with_icl(prompt, nlp, [{"input": text, "output": ""}])

    return response
def string_to_list(input_string):
    """
    Convert a string into a list using '\n' as the delimiter.
    Remove any empty strings from the resulting list.
    
    :param input_string: The input string to be converted
    :return: A list of non-empty strings
    """
    # Split the string by '\n' and remove any empty strings
    return [item.strip() for item in input_string.split('\n') if item.strip()]

# Example usage
sample_text = """
Hello, my name is John Doe. You can reach me at john.doe@example.com or
through my work email johnd@company.com. My colleague's email is
jane.smith@example.org.
"""


# Extract emails
extracted_emails = extract_emails(chil_info["authors"][0], llama3)
# Convert the extracted emails to a list
extracted_emails = string_to_list(extracted_emails)
print(extracted_emails)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


['zrr@bupt.edu.cn', 'euphy@bupt.edu.cn', 'cy_z_feng@bupt.edu.cn']


In [14]:
def extract_and_clean_emails(text, pipeline):
    prompt = f"""
    Extract all email addresses from the following text.
    Clean the extracted email addresses by removing any unnecessary characters or formatting issues.
    Output only the cleaned email addresses, one per line.
    
    Text: {text}
    
    Cleaned and extracted email addresses:
    """
    # Generate the response
    response = generate_text_with_icl(prompt, pipeline, [{"input": text, "output": ""}], max_new_tokens=256, temperature=0.00001, top_p=0.99)
    return response

def process_dataframe(pipeline, df: pd.DataFrame, text_column: str):
    # Extract and clean emails
    processed_emails = df[text_column].apply(lambda x: extract_and_clean_emails(x, pipeline))
    
    # Create a new dataframe with processed emails
    new_df = df.copy()
    new_df['processed_emails'] = processed_emails
    
    # Reorder columns to put processed_emails right after the original text column
    cols = list(new_df.columns)
    text_index = cols.index(text_column)
    cols.insert(text_index + 1, cols.pop(cols.index('processed_emails')))
    new_df = new_df[cols]
    
    return new_df
chil_info = process_dataframe(llama3, chil_info, "authors")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


zrr@bupt.edu.cn
euphy@bupt.edu.cn
cy_z_feng@bupt.edu.cn


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


haoran@cs.toronto.edu
amyxlu@cs.toronto.edu
msa@cs.toronto.edu
mmd@mit.edu
marzyeh@cs.toronto.edu


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

am2@stanford.edu
joerke@stanford.edu
wdenton@stanford.edu
barbarae@stanford.edu
ebrun@cs.stanford.edu


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


marikacusick@stanford.edu
gchertow@stanford.edu
owens@stanford.edu
micwilliams@stanfordhealthcare.org
sherrirose@stanford.edu


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


pkasl@ucsd.edu
ssoltani@ucsd.edu
lbruce@ucsd.edu
vkviswan@ucsd.edu
wendy.hartogensis@ucsf.edu
a1gupta@ucsd.edu
ialtintas@ucsd.edu
Stephan.Dilchert@baruch.cuny.edu
mt3qb@mst.edu
raosukru@msu.edu
stickn21@msu.edu
nadendla@mst.edu
canfieldci@mst.edu


In [8]:
import pandas as pd

def clean_titles(pipeline, df: pd.DataFrame, prompt: str):
    cleaned_titles = []
    for title in df["title"]:
        formatted_prompt = prompt.format(title=title)
        cleaned_title = generate_text_with_icl(formatted_prompt, pipeline, [{"input": formatted_prompt, "output": ""}], max_new_tokens=256, temperature=0.00001, top_p=0.99)
        print(cleaned_title)
        cleaned_titles.append(cleaned_title)
    return cleaned_titles

def process_dataframe(pipeline, df: pd.DataFrame, prompt: str):
    # Clean the titles
    cleaned_titles = clean_titles(pipeline, df, prompt)
    
    # Create a new dataframe with cleaned titles
    new_df = df.copy()
    new_df['cleaned_title'] = cleaned_titles
    
    # Reorder columns to put cleaned_title right after the original title
    cols = list(new_df.columns)
    title_index = cols.index('title')
    cols.insert(title_index + 1, cols.pop(cols.index('cleaned_title')))
    new_df = new_df[cols]
    
    return new_df
cleaned_chil_info = process_dataframe(llama3, chil_info, title_cleaning_prompt)

# Display the first 5 rows of the new dataframe
print(cleaned_chil_info.head())
# Optionally, save the new dataframe to a CSV file
cleaned_chil_info.to_csv("processed_data/cleaned_chil_extracted_info.csv", index=False)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


BMM-Net: Automatic Segmentation of Edema in Optical Coherence Tomography Based on Boundary Detection and Multi-Scale Network


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Hurtful Words: Quantifying Biases in Clinical Contextual Word Embeddings


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

Adaptive Interventions with User-Defined Goals for Health Behavior Change


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Algorithmic Changes Are Not Enough: Evaluating the Removal of Race Adjustment from the eGFR Equation


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Across-Study Analysis of Wearable Datasets and the Generalizability of Acute Illness Monitoring Models
Learning Social Fairness Preferences from Non-Expert Stakeholder Opinions in Kidney Placement
   year                                              title  \
0  2020  B M M- Net: Automatic Segmentationof Edemain O...   
1  2020  Hurtful Words: Quantifying Biasesin Clinical C...   
2  2020  Disease State Prediction From Single- Cell Dat...   
3  2020  Defining Admissible Rewardsfor High- Confidenc...   
4  2020  Population-aware Hierarchical Bayesian Domain ...   

                                       cleaned_title  \
0  BMM-Net: Automatic Segmentation of Edema in Op...   
1  Hurtful Words: Quantifying Biases in Clinical ...   
2  Disease State Prediction from Single-Cell Data...   
3  Defining Admissible Rewards for High-Confidenc...   
4  Population-Aware Hierarchical Bayesian Domain ...   

                                             authors  \
0  Ruru Zhang, Beijing University of 

### get select API calls and save their queries to json somewhere.

In [1]:
import pandas as pd
import json
import os
from serpapi import GoogleSearch
from time import sleep

def spaces_to_underscores(filename):
    return filename.replace(' ', '_')

def underscores_to_spaces(filename):
    return filename.replace('_', ' ')

def clean_filename(title):
    # Remove invalid characters and limit length
    clean = "".join(c for c in title if c.isalnum() or c in (' ', '-', '_')).rstrip()
    return clean[:100]  # Limit filename length

def rename_files_in_directory(directory, convert_function):
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            new_filename = convert_function(filename)
            old_path = os.path.join(directory, filename)
            new_path = os.path.join(directory, new_filename)
            os.rename(old_path, new_path)
            print(f"Renamed: {filename} -> {new_filename}")

def clean_existing_files(directory):
    rename_files_in_directory(directory, spaces_to_underscores)

def search_and_save_results(df, api_key, output_dir='serpapi'):
    os.makedirs(output_dir, exist_ok=True)

    for index, row in df.iterrows():
        cleaned_title = row['cleaned_title']
        filename = clean_filename(cleaned_title)
        filename = spaces_to_underscores(filename) + '.json'
        
        params = {
            "api_key": api_key,
            "engine": "google_scholar",
            "q": cleaned_title,
            "hl": "en"
        }
        
        try:
            search = GoogleSearch(params)
            results = search.get_dict()
            
            with open(os.path.join(output_dir, filename), 'w', encoding='utf-8') as f:
                json.dump(results, f, ensure_ascii=False, indent=4)
            
            print(f"Saved results for: {cleaned_title}")
            sleep(2)
        
        except Exception as e:
            print(f"Error processing {cleaned_title}: {str(e)}")
    
    print("Finished processing all titles.")

# Usage
api_key = "your_api_key_here"  # Replace with your actual API key
cleaned_chil_info = pd.read_csv("processed_data/cleaned_chil_extracted_info.csv")
cleaned_chil_info_subset = cleaned_chil_info.head(10)
clean_existing_files('serpapi') # save and then clean to underscores for reading

# Search and save results
# search_and_save_results(cleaned_chil_info_subset, api_key)
# Clean up existing files



# Example of converting back to spaces (if needed)
# rename_files_in_directory('serpapi', underscores_to_spaces)

### Scrape and record necessary data from downloaded json files

In [2]:
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import pandas as pd
import os

# Download necessary NLTK data
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

print("NLTK version:", nltk.__version__)
print("NLTK data path:", nltk.data.path)

def load_json_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

def extract_original_title(data):
    return data['search_parameters']['q']

def compare_titles(original_title, result_title):
    original_tokens = word_tokenize(original_title.lower())
    result_tokens = word_tokenize(result_title.lower())
    
    stop_words = set(stopwords.words('english'))
    original_tokens = [token for token in original_tokens if token.isalpha() and token not in stop_words]
    result_tokens = [token for token in result_tokens if token.isalpha() and token not in stop_words]
    overlapping_words = set(original_tokens) & set(result_tokens)
    return len(overlapping_words) >= 2

def extract_citations_from_file(file_path):
    data = load_json_data(file_path)
    original_title = extract_original_title(data)
    
    if 'organic_results' in data and len(data['organic_results']) > 0:
        result = data['organic_results'][0]
        
        # Compare titles before extracting information
        if compare_titles(original_title, result['title']):
            publication_info = result.get('publication_info', {})
            
            authors = publication_info.get('authors', [])
            author_names = [author.get('name', '') for author in authors]
            author_ids = [author.get('author_id', '') for author in authors]
            
            citation_info = {
                'original_title': original_title,
                'title': result.get('title', ''),
                'authors': ', '.join(author_names),
                'author_ids': ', '.join(author_ids),
                'publication': publication_info.get('summary', ''),
                'total_citations': result.get('inline_links', {}).get('cited_by', {}).get('total', 0),
                'pdf_link': next((r['link'] for r in result.get('resources', []) if r.get('file_format') == 'PDF'), None),
                'citation_link': result.get('inl ine_links', {}).get('cited_by', {}).get('link', '')
            }
            return citation_info
    return None

def process_directory_and_update_dataframe(directory_path, df):
    new_data = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.json'):
            file_path = os.path.join(directory_path, filename)
            citation_info = extract_citations_from_file(file_path)
            if citation_info:
                new_data.append(citation_info)
    
    new_df = pd.DataFrame(new_data)
    
    # Merge the new dataframe with the original one
    merged_df = df.copy()
    for col in new_df.columns:
        if col in merged_df.columns:
            merged_df[col] = new_df[col].combine_first(merged_df[col])
        else:
            merged_df[col] = new_df[col]
    
    return merged_df

# Usage
directory_path = "serpapi/chil"  # Replace with your actual directory path
cleaned_chil_info = pd.read_csv("processed_data/cleaned_chil_extracted_info.csv")
cleaned_chil_info_subset = cleaned_chil_info.head(10)

updated_df = process_directory_and_update_dataframe(directory_path, cleaned_chil_info_subset)

# Save the updated dataframe
updated_df.to_csv("processed_data/updated_chil_extracted_info_subset_with_comparison.csv", index=False)

print("Updated dataframe head:")
print(updated_df.head())
print("\nUpdated dataframe info:")
print(updated_df.info())

NLTK version: 3.8.1
NLTK data path: ['/home/johnwu3/nltk_data', '/home/johnwu3/miniconda3/envs/LLM/nltk_data', '/home/johnwu3/miniconda3/envs/LLM/share/nltk_data', '/home/johnwu3/miniconda3/envs/LLM/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']
Updated dataframe head:
   year                                              title  \
0  2020  Adverse drug reaction discovery from electroni...   
1  2020  BMM-Net: automatic segmentation of edema in op...   
2  2020  Deidentification of free-text medical records ...   
3  2020  Defining admissible rewards for high-confidenc...   
4  2020  Disease state prediction from single-cell data...   

                                 authors  \
0             W Zhang, P Peissig, D Page   
1                       HE, Z Ou, M Song   
2  AEW Johnson, L Bulgarelli, TJ Pollard   
3  N Prasad, B Engelhardt, F Doshi-Velez   
4               N Ravindra, A Sehanobish   

                   

In [22]:
updated_df.head(5)

,year,title,authors,processed_emails,cleaned_title,abstract,code_count,gitlab_count,zenodo_count,dataset_count,...,tuh_eeg_epilepsy_corpus_count,tuh_eeg_events_corpus_count,tuh_eeg_seizure_corpus_count,tuh_eeg_slowing_corpus_count,original_title,author_ids,publication,total_citations,pdf_link,citation_link
0,2020,Adverse drug reaction discovery from electroni...,"W Zhang, P Peissig, D Page",zrr@bupt.edu.cn\neuphy@bupt.edu.cn\ncy_z_feng@...,BMM-Net: Automatic Segmentation of Edema in Op...,Retinal effusions and cysts caused by the leak...,0,0,0,0,...,0,0,0,0,Adverse Drug Reaction Discovery from Electroni...,"ToHDHHsAAAAJ, h9soD-oAAAAJ, IlsuQ4oAAAAJ","W Zhang, Z Kuang, P Peissig, D Page - Proceedi...",8,https://dl.acm.org/doi/pdf/10.1145/3368555.338...,https://scholar.google.com/scholar?cites=10979...
1,2020,BMM-Net: automatic segmentation of edema in op...,"HE, Z Ou, M Song",haoran@cs.toronto.edu\namyxlu@cs.toronto.edu\n...,Hurtful Words: Quantifying Biases in Clinical ...,"In this work, we examine the extent to which e...",1,0,0,2,...,0,0,0,0,BMM-Net: Automatic Segmentation of Edema in Op...,"J4akh64AAAAJ, aYvRNFYAAAAJ, 6MRGS-4AAAAJ","R Zhang, J He, S Shi, HE, Z Ou, M Song - Proce...",1,https://dl.acm.org/doi/pdf/10.1145/3368555.338...,https://scholar.google.com/scholar?cites=54690...
2,2020,Deidentification of free-text medical records ...,"AEW Johnson, L Bulgarelli, TJ Pollard",neal.ravindra@yale.edu\narijit.sehanobish@yale...,Disease State Prediction from Single-Cell Data...,Single-cell RNA sequencing (scRNA-seq) has rev...,1,0,0,0,...,0,0,0,0,De-Identification of Free-Text Medical Records...,"66GCSz8AAAAJ, vKfiYkcAAAAJ, kd2pMFYAAAAJ","AEW Johnson, L Bulgarelli, TJ Pollard - Procee...",67,https://dl.acm.org/doi/pdf/10.1145/3368555.338...,https://scholar.google.com/scholar?cites=70993...
3,2020,Defining admissible rewards for high-confidenc...,"N Prasad, B Engelhardt, F Doshi-Velez",np6@princeton.edu\nbee@cs.princeton.edu\nfinal...,Defining Admissible Rewards for High-Confidenc...,A key impediment to reinforcement learning (RL...,0,0,0,1,...,0,0,0,0,Defining Admissible Rewards for High-Confidenc...,"P6PMzKAAAAAJ, VEGtG7YAAAAJ, hwQtFB0AAAAJ","N Prasad, B Engelhardt, F Doshi-Velez - Procee...",6,https://dl.acm.org/doi/pdf/10.1145/3368555.338...,https://scholar.google.com/scholar?cites=18316...
4,2020,Disease state prediction from single-cell data...,"N Ravindra, A Sehanobish",vishwalim@nyu.edu\nnabeel@nyu.edu\nrumi.chunar...,Population-Aware Hierarchical Bayesian Domain ...,While machine learning is rapidly being develo...,1,0,0,0,...,0,0,0,0,Disease State Prediction from Single-Cell Data...,"atVAWRMAAAAJ, MEby6-QAAAAJ","N Ravindra, A Sehanobish, JL Pappalardo… - Pro...",44,https://dl.acm.org/doi/pdf/10.1145/3368555.338...,https://scholar.google.com/scholar?cites=87006...


In [9]:
type(updated_df["author_ids"][0])

str

In [12]:
updated_df.head(10)

,year,title,cleaned_title,authors,abstract,code_count,gitlab_count,zenodo_count,dataset_count,mimic_count,...,tuh_eeg_epilepsy_corpus_count,tuh_eeg_events_corpus_count,tuh_eeg_seizure_corpus_count,tuh_eeg_slowing_corpus_count,original_title,author_ids,publication,total_citations,pdf_link,citation_link
0,2020,Adverse drug reaction discovery from electroni...,BMM-Net: Automatic Segmentation of Edema in Op...,"W Zhang, P Peissig, D Page",Retinal effusions and cysts caused by the leak...,0,0,0,0,0,...,0,0,0,0,Adverse Drug Reaction Discovery from Electroni...,"ToHDHHsAAAAJ, h9soD-oAAAAJ, IlsuQ4oAAAAJ","W Zhang, Z Kuang, P Peissig, D Page - Proceedi...",8,https://dl.acm.org/doi/pdf/10.1145/3368555.338...,https://scholar.google.com/scholar?cites=10979...
1,2020,BMM-Net: automatic segmentation of edema in op...,Hurtful Words: Quantifying Biases in Clinical ...,"HE, Z Ou, M Song","In this work, we examine the extent to which e...",1,0,0,2,1,...,0,0,0,0,BMM-Net: Automatic Segmentation of Edema in Op...,"J4akh64AAAAJ, aYvRNFYAAAAJ, 6MRGS-4AAAAJ","R Zhang, J He, S Shi, HE, Z Ou, M Song - Proce...",1,https://dl.acm.org/doi/pdf/10.1145/3368555.338...,https://scholar.google.com/scholar?cites=54690...
2,2020,Deidentification of free-text medical records ...,Disease State Prediction from Single-Cell Data...,"AEW Johnson, L Bulgarelli, TJ Pollard",Single-cell RNA sequencing (scRNA-seq) has rev...,1,0,0,0,0,...,0,0,0,0,De-Identification of Free-Text Medical Records...,"66GCSz8AAAAJ, vKfiYkcAAAAJ, kd2pMFYAAAAJ","AEW Johnson, L Bulgarelli, TJ Pollard - Procee...",67,https://dl.acm.org/doi/pdf/10.1145/3368555.338...,https://scholar.google.com/scholar?cites=70993...
3,2020,Defining admissible rewards for high-confidenc...,Defining Admissible Rewards for High-Confidenc...,"N Prasad, B Engelhardt, F Doshi-Velez",A key impediment to reinforcement learning (RL...,0,0,0,1,1,...,0,0,0,0,Defining Admissible Rewards for High-Confidenc...,"P6PMzKAAAAAJ, VEGtG7YAAAAJ, hwQtFB0AAAAJ","N Prasad, B Engelhardt, F Doshi-Velez - Procee...",6,https://dl.acm.org/doi/pdf/10.1145/3368555.338...,https://scholar.google.com/scholar?cites=18316...
4,2020,Disease state prediction from single-cell data...,Population-Aware Hierarchical Bayesian Domain ...,"N Ravindra, A Sehanobish",While machine learning is rapidly being develo...,1,0,0,0,0,...,0,0,0,0,Disease State Prediction from Single-Cell Data...,"atVAWRMAAAAJ, MEby6-QAAAAJ","N Ravindra, A Sehanobish, JL Pappalardo… - Pro...",44,https://dl.acm.org/doi/pdf/10.1145/3368555.338...,https://scholar.google.com/scholar?cites=87006...
5,2020,Hurtful words: quantifying biases in clinical ...,Interactive Hybrid Approach to Combine Machine...,"H Zhang, AX Lu, M Abdalla, M McDermott",Automated assessment of rehabilitation exercis...,0,0,0,0,0,...,0,0,0,0,Hurtful Words: Quantifying Biases in Clinical ...,"6aWRAPkAAAAJ, gQpYbRsAAAAJ, U8D2dlMAAAAJ, _V96...","H Zhang, AX Lu, M Abdalla, M McDermott… - proc...",176,https://dl.acm.org/doi/pdf/10.1145/3368555.338...,https://scholar.google.com/scholar?cites=15347...
6,2020,Interactive hybrid approach to combine machine...,Using SNOMED to Automate Clinical Concept Mapping,"MH Lee, DP Siewiorek, A Smailagic",The International Classification of Disease (I...,0,0,0,0,0,...,0,0,0,0,Interactive Hybrid Approach to Combine Machine...,"quDiEBkAAAAJ, B13SvmAAAAAJ, qMCGI94AAAAJ","MH Lee, DP Siewiorek, A Smailagic… - Proceedin...",35,https://dl.acm.org/doi/pdf/10.1145/3368555.338...,https://scholar.google.com/scholar?cites=10230...
7,2020,Population-aware hierarchical bayesian domain ...,Variational Learning of Individual Survival Di...,"V Mhasawade, NA Rehman, R Chunara",The abundance of modern health data provides m...,1,0,0,1,0,...,0,0,0,0,Population-Aware Hierarchical Bayesian Domain ...,"S64xxT0AAAAJ, Hrl6vaoAAAAJ, 7NhhkR8AAAAJ","V Mhasawade, NA Rehman, R Chunara - Proceeding...",10,https://dl.acm.org/doi/pdf/10.1145/3368555.338...,https://scholar.google.com/scholar?cites=20628...
8,2020,Variational

In [15]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# ... [Previous code remains unchanged] ...

def count_nan_titles(chil_df, ml4h_df, mlhc_df):
    """
    Counts the number of rows where the 'title' column is NaN for each dataframe.
    
    Args:
    chil_df, ml4h_df, mlhc_df: pandas DataFrames containing paper data
    
    Returns:
    dict: A dictionary with the counts of NaN titles for each conference
    """
    nan_titles = {
        'CHIL': chil_df['title'].isna().sum(),
        'ML4H': ml4h_df['title'].isna().sum(),
        'MLHC': mlhc_df['title'].isna().sum()
    }
    
    total_nan_titles = sum(nan_titles.values())
    total_papers = len(chil_df) + len(ml4h_df) + len(mlhc_df)
    
    nan_titles['Total'] = total_nan_titles
    nan_titles['Percentage'] = (total_nan_titles / total_papers) * 100
    
    return nan_titles

# Load the dataframes
chil_semantic_df = pd.read_csv("processed_data/chil_semantic_scholar_citations.csv")
ml4h_semantic_df = pd.read_csv("processed_data/ml4h_semantic_scholar_citations.csv")
mlhc_semantic_df = pd.read_csv("processed_data/mlhc_semantic_scholar_citations.csv")

# Count NaN citations
nan_citation_counts = count_nan_citations(chil_semantic_df, ml4h_semantic_df, mlhc_semantic_df)

# Print the NaN citation results
print("\nNaN Citation Counts:")
for conference, count in nan_citation_counts.items():
    if conference == 'Percentage':
        print(f"{conference}: {count:.2f}%")
    else:
        print(f"{conference}: {count}")

# Count uncleaned titles
uncleaned_title_counts = count_uncleaned_titles(chil_semantic_df, ml4h_semantic_df, mlhc_semantic_df)

# Print the uncleaned title results
print("\nUncleaned Title Counts:")
for conference, count in uncleaned_title_counts.items():
    if conference == 'Percentage':
        print(f"{conference}: {count:.2f}%")
    else:
        print(f"{conference}: {count}")

# Count NaN titles
nan_title_counts = count_nan_titles(chil_semantic_df, ml4h_semantic_df, mlhc_semantic_df)

# Print the NaN title results
print("\nNaN Title Counts:")
for conference, count in nan_title_counts.items():
    if conference == 'Percentage':
        print(f"{conference}: {count:.2f}%")
    else:
        print(f"{conference}: {count}")

# ... [Rest of the code remains unchanged] ...


NaN Citation Counts:
CHIL: 29
ML4H: 16
MLHC: 110
Total: 155
Percentage: 24.41%

Uncleaned Title Counts:
CHIL: 4
ML4H: 3
MLHC: 23
Total: 30
Percentage: 4.72%

NaN Title Counts:
CHIL: 4
ML4H: 3
MLHC: 23
Total: 30
Percentage: 4.72%


In [17]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import os

# ... [Previous functions remain unchanged] ...

# Load the dataframes
chil_semantic_df = pd.read_csv("processed_data/chil_semantic_scholar_citations.csv")
ml4h_semantic_df = pd.read_csv("processed_data/ml4h_semantic_scholar_citations.csv")
mlhc_semantic_df = pd.read_csv("processed_data/mlhc_semantic_scholar_citations.csv")

# Print original dataframe sizes
print("\nOriginal Dataframe Sizes:")
print(f"CHIL: {len(chil_semantic_df)}")
print(f"ML4H: {len(ml4h_semantic_df)}")
print(f"MLHC: {len(mlhc_semantic_df)}")

# Count NaN titles before removal
nan_title_counts = count_nan_titles(chil_semantic_df, ml4h_semantic_df, mlhc_semantic_df)

# Print the NaN title results before removal
print("\nNaN Title Counts (Before Removal):")
for conference, count in nan_title_counts.items():
    if conference == 'Percentage':
        print(f"{conference}: {count:.2f}%")
    else:
        print(f"{conference}: {count}")

# Remove rows with NaN titles
chil_semantic_df = chil_semantic_df.dropna(subset=['title'])
ml4h_semantic_df = ml4h_semantic_df.dropna(subset=['title'])
mlhc_semantic_df = mlhc_semantic_df.dropna(subset=['title'])

# Print new dataframe sizes
print("\nDataframe Sizes After Removing NaN Titles:")
print(f"CHIL: {len(chil_semantic_df)}")
print(f"ML4H: {len(ml4h_semantic_df)}")
print(f"MLHC: {len(mlhc_semantic_df)}")

# Recount NaN citations after removing rows with NaN titles
nan_citation_counts = count_nan_citations(chil_semantic_df, ml4h_semantic_df, mlhc_semantic_df)

# Print the new NaN citation results
print("\nNaN Citation Counts (After Removing NaN Titles):")
for conference, count in nan_citation_counts.items():
    if conference == 'Percentage':
        print(f"{conference}: {count:.2f}%")
    else:
        print(f"{conference}: {count}")

# Save cleaned dataframes to new CSV files
output_dir = "processed_data/cleaned"
os.makedirs(output_dir, exist_ok=True)

chil_semantic_df.to_csv(f"{output_dir}/chil_semantic_scholar_citations_cleaned.csv", index=False)
ml4h_semantic_df.to_csv(f"{output_dir}/ml4h_semantic_scholar_citations_cleaned.csv", index=False)
mlhc_semantic_df.to_csv(f"{output_dir}/mlhc_semantic_scholar_citations_cleaned.csv", index=False)

print("\nCleaned CSV files have been saved in the 'processed_data/cleaned' directory.")

# ... [Rest of the code remains unchanged] ...


Original Dataframe Sizes:
CHIL: 155
ML4H: 134
MLHC: 346

NaN Title Counts (Before Removal):
CHIL: 4
ML4H: 3
MLHC: 23
Total: 30
Percentage: 4.72%

Dataframe Sizes After Removing NaN Titles:
CHIL: 151
ML4H: 131
MLHC: 323

NaN Citation Counts (After Removing NaN Titles):
CHIL: 25
ML4H: 13
MLHC: 87
Total: 125
Percentage: 20.66%

Cleaned CSV files have been saved in the 'processed_data/cleaned' directory.


In [34]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import os

# ... [Previous functions remain unchanged] ...

def get_titles_with_nan_citations(df):
    """
    Returns a list of cleaned titles for papers with NaN citation counts.
    
    Args:
    df: pandas DataFrame containing paper data
    
    Returns:
    list: A list of cleaned titles for papers with NaN citation counts
    """
    return df[df['citation_count'].isna()]['cleaned_title'].tolist()

# Load the cleaned dataframes
chil_semantic_df = pd.read_csv("processed_data/specifically_cleaned/chil_semantic_scholar_citations.csv")
ml4h_semantic_df = pd.read_csv("processed_data/specifically_cleaned/ml4h_semantic_scholar_citations.csv")
mlhc_semantic_df = pd.read_csv("processed_data/specifically_cleaned/mlhc_semantic_scholar_citations.csv")

# Get lists of titles with NaN citations for each conference
chil_nan_citation_titles = get_titles_with_nan_citations(chil_semantic_df)
ml4h_nan_citation_titles = get_titles_with_nan_citations(ml4h_semantic_df)
mlhc_nan_citation_titles = get_titles_with_nan_citations(mlhc_semantic_df)

# Print the results
print("\nTitles with NaN citations:")
print(f"\nCHIL ({len(chil_nan_citation_titles)}):")
for title in chil_nan_citation_titles:  # Print first 5 for brevity
    print(f"- {title}")
# if len(chil_nan_citation_titles) > 5:
#     print("...")

print(f"\nML4H ({len(ml4h_nan_citation_titles)}):")
for title in ml4h_nan_citation_titles:  # Print first 5 for brevity
    print(f"- {title}")
# if len(ml4h_nan_citation_titles) > 5:
#     print("...")

print(f"\nMLHC ({len(mlhc_nan_citation_titles)}):")
for title in mlhc_nan_citation_titles:  # Print first 5 for brevity
    print(f"- {title}")
# if len(mlhc_nan_citation_titles) > 5:
#     print("...")

# ... [Rest of the code remains unchanged] ...


Titles with NaN citations:

CHIL (23):
- MIC-Extract: A Data Extraction, Preprocessing, and Representation Pipeline for
- Visual Che Xbert: Addressing the Discrepancy Between Radiology Report Labels and Image Labels
- Che X-Transfer: Performance and Parameter Efficiency of Image Net Models for Chest X-Ray Interpretation
- Interpretable Missing Values in Healthcare: Figure 7 - Impact of Father's Education on Infant Mortality Risk. Appendix A: Testing for MCAR with EBM: Case Study.
- Toward the Practical Utility of Federated Learning in the Medical Domain
- Evaluating Model Performance in Medical Datasets Over Time: A Snapshot into the State of Proceedings and the First 20 Papers That Came Up in the Radiology Medical Journal When Searching for the Keyword "Machine Learning" and Filtering for Papers from To
- Understanding and Predicting Environment Effects on Individuals with T2D: Appendix A - CGM Dataset. We include information on the range of values for each external factor in our dat

In [32]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import os
import re

# ... [Previous functions remain unchanged] ...

def specific_cleaning(df):
    """
    Performs specific cleaning operations on the dataframe.
    
    Args:
    df: pandas DataFrame containing paper data
    
    Returns:
    pandas DataFrame: The cleaned dataframe
    """
    # Titles to remove
    titles_to_remove = [
        "Supplementary",
        "Motivation for What Purpose Was the Dataset Created?",
        "Conference Proceedings",
        "Variable Total Age (Mean SD) 55.14",
        "There is no title to clean",
        "Machine Learning for Healthcare August 8-10",
        "Development of a Clinical Decision Tool and Protocol for Identification and Treatment",
        "Clinical Abstract Track",
        "To 32.9 in 1,"
    ]
    
    # Remove rows with specified titles
    pattern = '|'.join(map(re.escape, titles_to_remove))
    df = df[~df['cleaned_title'].str.contains(pattern, case=False, regex=True)]
    
    # Clean specific titles
    df['cleaned_title'] = df['cleaned_title'].replace({
        "EG to Text: Learning to Write Medical Reports from G Recordings": 
        "EEG to Text: Learning to Write Medical Reports from EEG Recordings"
    })
    
    # Remove "Preprint: Under Review" and associated numbers
    df['cleaned_title'] = df['cleaned_title'].apply(lambda x: re.sub(r'Preprint: Under Review \d+:\s*\d+–\d+,', '', x).strip())
    df['cleaned_title'] = df['cleaned_title'].apply(lambda x: re.sub(r'Preprint: Under Review \d+–\d+,?', '', x).strip())
    
    # Remove rows where the cleaned title is empty after cleaning
    df = df[df['cleaned_title'] != '']
    
    return df

# Load the cleaned dataframes
chil_semantic_df = pd.read_csv("processed_data/cleaned/chil_semantic_scholar_citations_cleaned.csv")
ml4h_semantic_df = pd.read_csv("processed_data/cleaned/ml4h_semantic_scholar_citations_cleaned.csv")
mlhc_semantic_df = pd.read_csv("processed_data/cleaned/mlhc_semantic_scholar_citations_cleaned.csv")

# Apply specific cleaning to each dataframe
print("Applying specific cleaning operations...")
chil_semantic_df = specific_cleaning(chil_semantic_df)
ml4h_semantic_df = specific_cleaning(ml4h_semantic_df)
mlhc_semantic_df = specific_cleaning(mlhc_semantic_df)

# Print the number of rows in each dataframe after cleaning
print(f"\nRows after specific cleaning:")
print(f"CHIL: {len(chil_semantic_df)}")
print(f"ML4H: {len(ml4h_semantic_df)}")
print(f"MLHC: {len(mlhc_semantic_df)}")

# Save the specifically cleaned dataframes
output_dir = "processed_data/specifically_cleaned"
os.makedirs(output_dir, exist_ok=True)

chil_semantic_df.to_csv(f"{output_dir}/chil_semantic_scholar_citations.csv", index=False)
ml4h_semantic_df.to_csv(f"{output_dir}/ml4h_semantic_scholar_citations.csv", index=False)
mlhc_semantic_df.to_csv(f"{output_dir}/mlhc_semantic_scholar_citations.csv", index=False)

print(f"\nSpecifically cleaned CSV files have been saved in the '{output_dir}' directory.")

# Function to check for "preprint" in titles
def count_preprint_titles(df):
    return df['cleaned_title'].str.lower().str.contains('preprint').sum()

# Count and print the number of titles containing "preprint"
print("\nNumber of titles containing 'preprint':")
print(f"CHIL: {count_preprint_titles(chil_semantic_df)}")
print(f"ML4H: {count_preprint_titles(ml4h_semantic_df)}")
print(f"MLHC: {count_preprint_titles(mlhc_semantic_df)}")

# Function to get titles containing "preprint"
def get_preprint_titles(df):
    return df[df['cleaned_title'].str.lower().str.contains('preprint')]['cleaned_title'].tolist()

# Print any titles that still contain "preprint"
for name, df in [("CHIL", chil_semantic_df), ("ML4H", ml4h_semantic_df), ("MLHC", mlhc_semantic_df)]:
    preprint_titles = get_preprint_titles(df)
    if preprint_titles:
        print(f"\n{name} title(s) still containing 'preprint':")
        for title in preprint_titles:
            print(f"- {title}")
    else:
        print(f"\nNo {name} titles contain 'preprint'.")

# ... [Rest of the code remains unchanged] ...

Applying specific cleaning operations...

Rows after specific cleaning:
CHIL: 148
ML4H: 127
MLHC: 283

Specifically cleaned CSV files have been saved in the 'processed_data/specifically_cleaned' directory.

Number of titles containing 'preprint':
CHIL: 0
ML4H: 0
MLHC: 0

No CHIL titles contain 'preprint'.

No ML4H titles contain 'preprint'.

No MLHC titles contain 'preprint'.


/tmp/ipykernel_85908/566563017.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_title'] = df['cleaned_title'].replace({
/tmp/ipykernel_85908/566563017.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_title'] = df['cleaned_title'].apply(lambda x: re.sub(r'Preprint: Under Review \d+:\s*\d+–\d+,', '', x).strip())
/tmp/ipykernel_85908/566563017.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [36]:
import pandas as pd
import json
import os
from serpapi import GoogleSearch
from time import sleep
import re

def clean_filename(title):
    # Remove invalid characters and limit length
    clean = re.sub(r'[^\w\s-]', '', title).strip()
    return clean[:100]  # Limit filename length

def search_and_save_results(df, api_key, output_dir='serpapi_results'):
    os.makedirs(output_dir, exist_ok=True)
    
    # Filter for rows where citation_count is NaN
    df_nan_citations = df[pd.isna(df['citation_count'])]
    
    print(f"Processing {len(df_nan_citations)} titles with NaN citation counts...")
    
    for index, row in df_nan_citations.iterrows():
        cleaned_title = row['cleaned_title']
        filename = clean_filename(cleaned_title)
        filename = filename.replace(' ', '_') + '.json'
        
        params = {
            "api_key": api_key,
            "engine": "google_scholar",
            "q": cleaned_title,
            "hl": "en"
        }
        
        try:
            search = GoogleSearch(params)
            results = search.get_dict()
            
            with open(os.path.join(output_dir, filename), 'w', encoding='utf-8') as f:
                json.dump(results, f, ensure_ascii=False, indent=4)
            
            print(f"Saved results for: {cleaned_title}")
            sleep(2)  # To avoid hitting rate limits
        
        except Exception as e:
            print(f"Error processing {cleaned_title}: {str(e)}")
    
    print("Finished processing all titles with NaN citation counts.")

# Usage
api_key = ""  # Replace with your actual API key

# Load the cleaned dataframes
input_dir = "processed_data/specifically_cleaned"
chil_df = pd.read_csv(f"{input_dir}/chil_semantic_scholar_citations.csv")
ml4h_df = pd.read_csv(f"{input_dir}/ml4h_semantic_scholar_citations.csv")
mlhc_df = pd.read_csv(f"{input_dir}/mlhc_semantic_scholar_citations.csv")

# Process each dataframe
for name, df in [("CHIL", chil_df), ("ML4H", ml4h_df), ("MLHC", mlhc_df)]:
    print(f"\nProcessing {name} titles...")
    output_dir = f"serpapi_results_{name.lower()}_nan_citations"
    search_and_save_results(df, api_key, output_dir)

print("All processing completed.")


Processing CHIL titles...
Processing 23 titles with NaN citation counts...
Saved results for: MIC-Extract: A Data Extraction, Preprocessing, and Representation Pipeline for
Saved results for: Visual Che Xbert: Addressing the Discrepancy Between Radiology Report Labels and Image Labels
Saved results for: Che X-Transfer: Performance and Parameter Efficiency of Image Net Models for Chest X-Ray Interpretation
Saved results for: Interpretable Missing Values in Healthcare: Figure 7 - Impact of Father's Education on Infant Mortality Risk. Appendix A: Testing for MCAR with EBM: Case Study.
Saved results for: Toward the Practical Utility of Federated Learning in the Medical Domain
Saved results for: Evaluating Model Performance in Medical Datasets Over Time: A Snapshot into the State of Proceedings and the First 20 Papers That Came Up in the Radiology Medical Journal When Searching for the Keyword "Machine Learning" and Filtering for Papers from To
Saved results for: Understanding and Predicti

In [1]:
import pandas as pd
import json
import os
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

# Download necessary NLTK data
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

def clean_filename(title):
    # Remove special characters and limit length
    clean = re.sub(r'[^\w\s-]', '', title).strip()
    clean = re.sub(r'\s+', '_', clean)  # Replace spaces with underscores
    return clean[:100]  # Limit filename length

def compare_titles(original_title, result_title):
    original_tokens = word_tokenize(original_title.lower())
    result_tokens = word_tokenize(result_title.lower())
    stop_words = set(stopwords.words('english'))
    original_tokens = [token for token in original_tokens if token.isalpha() and token not in stop_words]
    result_tokens = [token for token in result_tokens if token.isalpha() and token not in stop_words]
    overlapping_words = set(original_tokens) & set(result_tokens)
    return len(overlapping_words) >= 3

def extract_citations_from_file(file_path, original_title):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    if 'organic_results' in data and len(data['organic_results']) > 0:
        result = data['organic_results'][0]
        if compare_titles(original_title, result['title']):
            return result.get('inline_links', {}).get('cited_by', {}).get('total', 0)
    return None

def process_dataframe(df, input_dir):
    # Filter for rows where citation_count is NaN
    df_nan_citations = df[pd.isna(df['citation_count'])]
    
    print(f"Processing {len(df_nan_citations)} titles with NaN citation counts...")
    unmatched_count = 0
    for index, row in df_nan_citations.iterrows():
        cleaned_title = row['cleaned_title']
        filename = clean_filename(cleaned_title) + '.json'
        file_path = os.path.join(input_dir, filename)
        
        if os.path.exists(file_path):
            # Extract citations
            citations = extract_citations_from_file(file_path, cleaned_title)
            if citations is not None:
                df.loc[index, 'citation_count'] = citations
            else:
                unmatched_count += 1
        else:
            print(f"File not found for: {cleaned_title}")
            unmatched_count += 1
    
    # Drop rows with NaN citation counts
    df_cleaned = df.dropna(subset=['citation_count'])
    
    return df_cleaned, unmatched_count

# Usage
# Load the cleaned dataframes
input_dir = "processed_data/specifically_cleaned"
chil_df = pd.read_csv(f"{input_dir}/chil_semantic_scholar_citations.csv")
ml4h_df = pd.read_csv(f"{input_dir}/ml4h_semantic_scholar_citations.csv")
mlhc_df = pd.read_csv(f"{input_dir}/mlhc_semantic_scholar_citations.csv")

# Process each dataframe
for name, df in [("CHIL", chil_df), ("ML4H", ml4h_df), ("MLHC", mlhc_df)]:
    print(f"\nProcessing {name} titles...")
    input_dir = f"serpapi_results_{name.lower()}_nan_citations"
    df_cleaned, unmatched_count = process_dataframe(df, input_dir)
    
    # Save the cleaned dataframe
    output_file = f"processed_data/cleaned_{name.lower()}_citations.csv"
    df_cleaned.to_csv(output_file, index=False)
    print(f"Saved cleaned {name} dataframe to {output_file}")
    
    # Print statistics
    print(f"Total entries processed: {len(df)}")
    print(f"Entries with NaN citations: {len(df) - len(df_cleaned)}")
    print(f"Entries without matching SerpAPI results: {unmatched_count}")
    print(f"Entries with updated citations: {len(df_cleaned) - (len(df) - len(df[pd.isna(df['citation_count'])]))}")

print("All processing completed.")


Processing CHIL titles...
Processing 23 titles with NaN citation counts...
Saved cleaned CHIL dataframe to processed_data/cleaned_chil_citations.csv
Total entries processed: 148
Entries with NaN citations: 1
Entries without matching SerpAPI results: 1
Entries with updated citations: 0

Processing ML4H titles...
Processing 9 titles with NaN citation counts...
Saved cleaned ML4H dataframe to processed_data/cleaned_ml4h_citations.csv
Total entries processed: 127
Entries with NaN citations: 4
Entries without matching SerpAPI results: 4
Entries with updated citations: 0

Processing MLHC titles...
Processing 60 titles with NaN citation counts...
Saved cleaned MLHC dataframe to processed_data/cleaned_mlhc_citations.csv
Total entries processed: 283
Entries with NaN citations: 25
Entries without matching SerpAPI results: 25
Entries with updated citations: 0
All processing completed.
